# Notebook for visualizing how backward pass is performed step by step through different layers with comparison with torch autograd

In [34]:
from abc import ABC, abstractmethod
import copy
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

from typing import Union, Tuple
from IPython.display import HTML, clear_output

In [2]:
BOS, EOS = ' ', '\n'

data = pd.read_json("./arxivData.json")
lines = data.apply(lambda row: (row['title'] + ' ; ' + row['summary'])[:128], axis=1) \
            .apply(lambda line: BOS + line.replace(EOS, ' ') + EOS) \
            .tolist()

tokens = list(set(''.join(lines)))

num_tokens = len(tokens)
print('num_tokens = ', num_tokens)

token_to_id = {token: idx for idx, token in enumerate(tokens)}

def to_matrix(data, token_to_id, max_len=None, dtype='int32', batch_first=True):
    """Casts a list of names into rnn-digestable matrix"""
    
    max_len = max_len or max(map(len, data))
    data_ix = np.zeros([len(data), max_len], dtype) + token_to_id[' ']

    for i in range(len(data)):
        line_ix = [token_to_id[c] for c in data[i]]
        data_ix[i, :len(line_ix)] = line_ix
        
    if not batch_first: # convert [batch, time] into [time, batch]
        data_ix = np.transpose(data_ix)

    return data_ix

num_tokens =  136


In [3]:
class BaseLayer(ABC):
    @abstractmethod
    def __init__(self) -> None:
        pass

    def __call__(self, x: np.array, grad: bool = True) -> np.array:
        return self.forward(x, grad)

    @abstractmethod
    def forward(self, x: np.array, grad: bool = True) -> np.array:
        pass

    @abstractmethod
    def backward(self, output_error: np.array) -> np.array:
        pass

# emb check

In [4]:
class Embedding(BaseLayer):
    def __init__(self, n_input, emb_dim, pad_idx=None):
        self.n_input = n_input
        self.emb_dim = emb_dim
        self.pad_idx = pad_idx
        
        self.weights = np.random.normal(scale=np.sqrt(2/(n_input+emb_dim)), size=(n_input, emb_dim))

    def set_optimizer(self, optimizer):
        self.weights_optimizer = copy.copy(optimizer)

        self.weights_optimizer.set_weight(self.weights)

    def forward(self, x, grad=True):
        self.input = x
        return self.weights[x]

    def backward(self, output_error):
        weights_grad = np.zeros_like(self.weights)
        input_shape_len = len(self.input.shape)

        if input_shape_len == 2:
            for batch_n, s in enumerate(self.input):
                for i, emb_i in enumerate(s):
                    weights_grad[emb_i] += output_error[batch_n][i]

        elif input_shape_len == 1:
            for i, emb_i in enumerate(self.input):
                weights_grad[emb_i] += output_error[i]

        if self.pad_idx is not None:
            weights_grad[self.pad_idx] = 0

        # self.weights = self.weights_optimizer.step(weights_grad)
        return weights_grad

In [5]:
# проверка на то, что градиент эмбеддингов считается правильно
sample = to_matrix(np.random.choice(lines, size=5), token_to_id, max_len=130)
emb = Embedding(len(token_to_id), 16)
print("Embeddings shape:", emb.weights.shape)

torch_emb = torch.nn.Embedding(len(token_to_id), 16)
torch_emb.weight.data = torch.as_tensor(emb.weights)

torch_out = torch_emb(torch.as_tensor(sample))

print("Forward совпадает:", np.allclose(torch_out.detach().numpy(), emb(sample)))

check_error = np.random.normal(0, 100, torch_out.shape)
check_error_torch = torch.tensor(check_error)

torch_out.backward(check_error_torch)

print("Градиенты совпадают", np.allclose(torch_emb.weight.grad.detach().numpy(), emb.backward(check_error)))

Embeddings shape: (136, 16)
Forward совпадает: True
Градиенты совпадают True


# conv1d check

In [6]:
class Conv1dVanilla(BaseLayer):
    """
    Сверточный слой, со страйдом 1 и без паддингов, для 1 предложения, не батча
    """
    def __init__(self, in_channels, out_channels, kernel_size):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size

        scale = np.sqrt(1/(in_channels*kernel_size))
        self.kernel = np.random.uniform(-scale, scale, size=(out_channels, in_channels, kernel_size))
        self.bias = np.random.uniform(-scale, scale, size=(out_channels))

    def set_optimizer(self, optimizer):
        self.kernel_optimizer = copy.copy(optimizer)
        self.bias_optimizer = copy.copy(optimizer)

        self.kernel_optimizer.set_weight(self.kernel)
        self.bias_optimizer.set_weight(self.bias)

    def forward(self, x, grad=True):
        self.input = x

        self.output_len = x.shape[0] - self.kernel_size + 1
        output = np.zeros(shape=(self.output_len, self.out_channels))

        for kernel_i, ker in enumerate(self.kernel):
            for i in range(self.output_len):
                output[i:self.kernel_size+i, kernel_i] = self.bias[kernel_i] + np.sum(x[i:self.kernel_size+i, :] * ker.T)

        return output

    def backward(self, output_error):
        dy_dkernel = np.zeros(shape=self.kernel.shape)
        dy_dbias = np.zeros(shape=self.bias.shape)
        dy_dx = np.zeros(shape=self.input.shape)

        for kernel_i, ker in enumerate(self.kernel):
            helper_k = np.zeros(shape=ker.T.shape)

            for i in range(self.output_len):
                helper_k += self.input[i:self.kernel_size+i, :] * output_error[i, kernel_i]
                dy_dx[i:self.kernel_size+i, :] += ker.T * output_error[i, kernel_i]

            dy_dkernel[kernel_i] = helper_k.T
            dy_dbias[kernel_i] = np.sum(output_error[:, kernel_i])

        # self.kernel = self.kernel_optimizer.step(dy_dkernel)
        # self.bias = self.bias_optimizer.step(dy_dbias)

        # return dy_dx
        return dy_dkernel, dy_dbias, dy_dx

In [7]:
sample = to_matrix(np.random.choice(lines, size=5), token_to_id, max_len=130)

emb = Embedding(len(token_to_id), 16)
encoded_data = emb(sample[0])

conv = Conv1dVanilla(16, 4, 5)
torch_conv = torch.nn.Conv1d(16, 4, 5)
torch_conv.weight.data = torch.as_tensor(conv.kernel)
torch_conv.bias.data = torch.as_tensor(conv.bias)

torch_input = torch.tensor(encoded_data[np.newaxis, :], dtype=torch.float64, requires_grad=True)
torch_out = torch_conv(torch_input.permute(0, 2, 1))  # permute потому что torch на вход принимает формат [BATCH_SIZE, EMB_DIM, SENTENCE_LEN]

# проверка что forward работает также как у torch, 
print("Vanilla Forward такой же как и torch forward:", np.allclose(torch_out.permute(0, 2, 1).detach().numpy(), conv(encoded_data)))

print("Shape выхода свертки:", conv(encoded_data).shape)

# случайная ошибка, которая приходит "сверху" от вышестоящих слоев, по размеру она совпадает с выходом слоя
check_error = np.random.normal(loc=-3, scale=100, size=(conv(encoded_data).shape))
check_error_torch = torch.tensor(np.transpose(check_error[np.newaxis, :], (0, 2, 1)))

torch_out.backward(check_error_torch)  # считаем градиенты для всех тензоров, которые участвуют в forward проходе
kernel_grad, bias_grad, in_error = conv.backward(check_error)  # тоже самое, только в ручном слое

# проверка градиентов весов и входа
print("Градиент по входу совпадает:", np.allclose(torch_input.grad.detach().numpy(), in_error))
print("Градиент по смещениям совпадает:", np.allclose(torch_conv.bias.grad.detach().numpy(), bias_grad))
print("Градиент по ядру совпадает:", np.allclose(torch_conv.weight.grad.detach().numpy(), kernel_grad))

Vanilla Forward такой же как и torch forward: True
Shape выхода свертки: (126, 4)
Градиент по входу совпадает: True
Градиент по смещениям совпадает: True
Градиент по ядру совпадает: True


In [8]:
class Conv1d(BaseLayer):
    """
    Сверточный слой, со страйдом 1 и без паддингов, для батча
    """
    def __init__(self, in_channels, out_channels, kernel_size):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size

        scale = np.sqrt(1/(in_channels*kernel_size))
        self.kernel = np.random.uniform(-scale, scale, size=(out_channels, in_channels, kernel_size))
        self.bias = np.random.uniform(-scale, scale, size=(out_channels))

    def set_optimizer(self, optimizer):
        self.kernel_optimizer = copy.copy(optimizer)
        self.bias_optimizer = copy.copy(optimizer)

        self.kernel_optimizer.set_weight(self.kernel)
        self.bias_optimizer.set_weight(self.bias)

    def forward(self, x, grad=True):
        """
        Работает с битчами вида [BATCH_SIZE, SENTENCE_LEN, EMB_DIM]
        """
        self.input = x
        self.batch_size = x.shape[0]
        self.input_len = x.shape[1]
        self.output_len = self.input_len - self.kernel_size + 1

        result = []

        for sentence in x:
            result.append(self._forward_for_one(sentence))

        return np.array(result)

    def _forward_for_one(self, x):
        """
        Просто свертка для 1 предложения
        """
        output = np.zeros(shape=(self.output_len, self.out_channels))

        # для каждого выходного канала и ядра, отвечающего за этот канал
        for kernel_i, ker in enumerate(self.kernel):
            # по выходной длине
            for i in range(self.output_len):
                # умножаем срез по размеру ядра на ядро и суммируем
                output[i:self.kernel_size+i, kernel_i] = self.bias[kernel_i] + np.sum(x[i:self.kernel_size+i, :] * ker.T)

        return output

    def backward(self, output_error):
        """
        Градиенты по всем батчу
        """
        dy_dkernels = []
        dy_dbiass = []
        dy_dxs = []

        for i in range(self.batch_size):
            dy_dkernel, dy_dbias, dy_dx = self._calc_grad_for_one(output_error[i], self.input[i])
            dy_dkernels.append(dy_dkernel)
            dy_dbiass.append(dy_dbias)
            dy_dxs.append(dy_dx)

        dy_dkernels = np.sum(np.array(dy_dkernels), axis=0)  # суммируем градиенты по батчу
        dy_dbiass = np.sum(np.array(dy_dbiass), axis=0)
        dy_dxs = np.array(dy_dxs)

        # self.kernel = self.kernel_optimizer.step(dy_dkernels)  # делаем шаг спуска по сумме градиентов
        # self.bias = self.bias_optimizer.step(dy_dbiass)

        # return dy_dxs
        return dy_dkernels, dy_dbiass, dy_dxs

    def _calc_grad_for_one(self, output_error, x):
        dy_dkernel = np.zeros(shape=self.kernel.shape)
        dy_dbias = np.zeros(shape=self.bias.shape)
        dy_dx = np.zeros(shape=x.shape)

        for kernel_i, ker in enumerate(self.kernel):
            helper_k = np.zeros(shape=ker.T.shape)

            for i in range(self.output_len):
                helper_k += x[i:self.kernel_size+i, :] * output_error[i, kernel_i]
                dy_dx[i:self.kernel_size+i, :] += ker.T * output_error[i, kernel_i]

            dy_dkernel[kernel_i] = helper_k.T
            dy_dbias[kernel_i] = np.sum(output_error[:, kernel_i])

        return dy_dkernel, dy_dbias, dy_dx

In [9]:
# тоже самое, только теперь ручная свертка умеет работать с батчами, np.newaxis теперь не нужен, размер входа будет [BATCH_SIZE, SENTENCE_LEN, EMB_DIM]

sample = to_matrix(np.random.choice(lines, size=5), token_to_id, max_len=130)

emb = Embedding(len(token_to_id), 16)
encoded_data = emb(sample) # батч из 5 предложений

conv = Conv1d(16, 4, 5)
torch_conv = torch.nn.Conv1d(16, 4, 5)
torch_conv.weight.data = torch.as_tensor(conv.kernel)
torch_conv.bias.data = torch.as_tensor(conv.bias)

torch_input = torch.tensor(encoded_data, dtype=torch.float64, requires_grad=True)
torch_out = torch_conv(torch_input.permute(0, 2, 1))  # permute потому что torch на вход принимает формат [BATCH_SIZE, EMB_DIM, SENTENCE_LEN]

# проверка что forward работает также как у torch, 
print("Vanilla Forward такой же как и torch forward:", np.allclose(torch_out.permute(0, 2, 1).detach().numpy(), conv(encoded_data)))

print("Shape выхода свертки:", conv(encoded_data).shape)

# случайная ошибка, которая приходит "сверху" от вышестоящих слоев, по размеру она совпадает с выходом слоя
check_error = np.random.normal(loc=-3, scale=100, size=(conv(encoded_data).shape))
check_error_torch = torch.tensor(np.transpose(check_error, (0, 2, 1)))

torch_out.backward(check_error_torch)  # считаем градиенты для всех тензоров, которые участвуют в forward проходе
kernel_grad, bias_grad, in_error = conv.backward(check_error)  # тоже самое, только в ручном слое

# проверка градиентов весов и входа
print("Градиент по входу совпадает:", np.allclose(torch_input.grad.detach().numpy(), in_error))
print("Градиент по смещениям совпадает:", np.allclose(torch_conv.bias.grad.detach().numpy(), bias_grad))
print("Градиент по ядру совпадает:", np.allclose(torch_conv.weight.grad.detach().numpy(), kernel_grad))

Vanilla Forward такой же как и torch forward: True
Shape выхода свертки: (5, 126, 4)
Градиент по входу совпадает: True
Градиент по смещениям совпадает: True
Градиент по ядру совпадает: True


# linear check

In [10]:
class Linear(BaseLayer):
    """
    Linear class permorms ordinary FC layer in neural networks
    Parameters:
    n_input - size of input neurons
    n_output - size of output neurons
    Methods:
    set_optimizer(optimizer) - is used for setting an optimizer for gradient descent
    forward(x) - performs forward pass of the layer
    backward(output_error, learning_rate) - performs backward pass of the layer
    """

    def __init__(self, n_input: int, n_output: int) -> None:
        super().__init__()
        self.input = None
        self.n_input = n_input
        self.n_output = n_output
        self.w = np.random.normal(scale=np.sqrt(2 / (n_input + n_output)), size=(n_input, n_output))
        self.b = np.random.normal(scale=np.sqrt(2 / (n_input + n_output)), size=(1, n_output))

        self.w_optimizer = None
        self.b_optimizer = None

    def set_optimizer(self, optimizer) -> None:
        self.w_optimizer = copy.copy(optimizer)
        self.b_optimizer = copy.copy(optimizer)

        self.w_optimizer.set_weight(self.w)
        self.b_optimizer.set_weight(self.b)

    def forward(self, x: np.array, grad: bool = True) -> np.array:
        self.input = x
        return x.dot(self.w) + self.b

    def backward(self, output_error: np.array) -> np.array:
        # assert self.w_optimizer is not None and self.b_optimizer is not None, 'You should set an optimizer'
        w_grad = self.input.T.dot(output_error)
        b_grad = np.ones((1, len(output_error))).dot(output_error)
        input_error = output_error.dot(self.w.T)

        # self.w = self.w_optimizer.step(w_grad)
        # self.b = self.b_optimizer.step(b_grad)
        return w_grad, b_grad, input_error

In [11]:
# проверка того, что линейный слой работает правильно, транспонирование весов происходит потому, что домножение на веса в моем слое справа

sample = np.random.normal(loc=0, scale=100, size=(100, 16))

linear = Linear(16, 25)
torch_linear = torch.nn.Linear(16, 25)
torch_linear.weight.data = torch.as_tensor(linear.w.T)
torch_linear.bias.data = torch.as_tensor(linear.b)

torch_input = torch.tensor(sample, dtype=torch.float64, requires_grad=True)
torch_out = torch_linear(torch_input)

# проверка что forward работает также как у torch, 
print("Vanilla Forward такой же как и torch forward:", np.allclose(torch_out.detach().numpy(), linear(sample)))

print("Shape выхода линейного слоя:", linear(sample).shape)

# случайная ошибка, которая приходит "сверху" от вышестоящих слоев, по размеру она совпадает с выходом слоя
check_error = np.random.normal(loc=-3, scale=100, size=(linear(sample).shape))
check_error_torch = torch.tensor(check_error)

torch_out.backward(check_error_torch)  # считаем градиенты для всех тензоров, которые участвуют в forward проходе
kernel_grad, bias_grad, in_error = linear.backward(check_error)  # тоже самое, только в ручном слое

# проверка градиентов весов и входа
print("Градиент по входу совпадает:", np.allclose(torch_input.grad.detach().numpy(), in_error))
print("Градиент по смещениям совпадает:", np.allclose(torch_linear.bias.grad.detach().numpy(), bias_grad))
print("Градиент по ядру совпадает:", np.allclose(torch_linear.weight.grad.detach().numpy(), kernel_grad.T))

Vanilla Forward такой же как и torch forward: True
Shape выхода линейного слоя: (100, 25)
Градиент по входу совпадает: True
Градиент по смещениям совпадает: True
Градиент по ядру совпадает: True


In [12]:
class Linear3d(BaseLayer):
    """
    Linear class permorms ordinary FC layer in neural networks
    Parameters:
    n_input - size of input neurons
    n_output - size of output neurons
    Methods:
    set_optimizer(optimizer) - is used for setting an optimizer for gradient descent
    forward(x) - performs forward pass of the layer
    backward(output_error, learning_rate) - performs backward pass of the layer
    """

    def __init__(self, n_input: int, n_output: int) -> None:
        super().__init__()
        self.input = None
        self.n_input = n_input
        self.n_output = n_output
        self.w = np.random.normal(scale=np.sqrt(2 / (n_input + n_output)), size=(n_input, n_output))
        self.b = np.random.normal(scale=np.sqrt(2 / (n_input + n_output)), size=(1, n_output))

        self.w_optimizer = None
        self.b_optimizer = None

    def set_optimizer(self, optimizer) -> None:
        self.w_optimizer = copy.copy(optimizer)
        self.b_optimizer = copy.copy(optimizer)

        self.w_optimizer.set_weight(self.w)
        self.b_optimizer.set_weight(self.b)

    def forward(self, x: np.array, grad: bool = True) -> np.array:
        self.input = x
        return np.matmul(x, self.w) + self.b  # the same as @

    def backward(self, output_error: np.array) -> np.array:
        # assert self.w_optimizer is not None and self.b_optimizer is not None, 'You should set an optimizer'
        # перемножаем последние 2 измерения друг с другом с помощью matmul и суммируем
        w_grad = np.sum(np.transpose(self.input, (0, 2, 1)) @ output_error, axis=0)
        b_grad = np.sum(output_error, axis=(0, 1))
        input_error = output_error @ self.w.T

        # self.w = self.w_optimizer.step(w_grad)
        # self.b = self.b_optimizer.step(b_grad)
        return w_grad, b_grad, input_error

In [13]:
# проверка того, что линейный слой для 3-х измерений работает правильно, транспонирование весов происходит потому, что домножение на веса в моем слое справа

sample = np.random.normal(loc=0, scale=100, size=(100, 32, 16))

linear = Linear3d(16, 25)
torch_linear = torch.nn.Linear(16, 25)
torch_linear.weight.data = torch.as_tensor(linear.w.T)
torch_linear.bias.data = torch.as_tensor(linear.b)

torch_input = torch.tensor(sample, dtype=torch.float64, requires_grad=True)
torch_out = torch_linear(torch_input)

# проверка что forward работает также как у torch, 
print("Vanilla Forward такой же как и torch forward:", np.allclose(torch_out.detach().numpy(), linear(sample)))

print("Shape выхода линейного слоя:", linear(sample).shape)

# случайная ошибка, которая приходит "сверху" от вышестоящих слоев, по размеру она совпадает с выходом слоя
check_error = np.random.normal(loc=-3, scale=100, size=(linear(sample).shape))
check_error_torch = torch.tensor(check_error)

torch_out.backward(check_error_torch)  # считаем градиенты для всех тензоров, которые участвуют в forward проходе
kernel_grad, bias_grad, in_error = linear.backward(check_error)  # тоже самое, только в ручном слое

# проверка градиентов весов и входа
print("Градиент по входу совпадает:", np.allclose(torch_input.grad.detach().numpy(), in_error))
print("Градиент по смещениям совпадает:", np.allclose(torch_linear.bias.grad.detach().numpy(), bias_grad))
print("Градиент по ядру совпадает:", np.allclose(torch_linear.weight.grad.detach().numpy(), kernel_grad.T))

Vanilla Forward такой же как и torch forward: True
Shape выхода линейного слоя: (100, 32, 25)
Градиент по входу совпадает: True
Градиент по смещениям совпадает: True
Градиент по ядру совпадает: True


In [14]:
np.transpose(sample, (0, 2, 1))[1].dot(check_error[1])

array([[  67568.71473288,  -54068.69574521,   23596.48727212,
        -128521.19845805,    9880.1331249 ,  -36268.42439887,
         -41735.56494009,   56111.89244778,   35586.33672801,
         -34324.09351658,  -32552.19429502,   57874.73568831,
          47993.05864908,   -7400.9907628 ,  -56195.24721095,
         -68185.55195928,  -31738.97832506,   49116.0314091 ,
          -9851.24133631,  -53612.86705739,  -41753.73630245,
         -19076.15984663,   81432.55214513,  -24529.29526251,
          77003.18886427],
       [  17865.53255925,   -5576.68223925,   63146.60126128,
         -22283.36554166,   23351.3610662 ,  -57360.81541882,
         -45170.46704203,  -18070.80811844,   41350.80997168,
          31992.24958086,   87390.00853691,  -10115.6751183 ,
           2225.1500625 ,   22179.83199527,    6592.1886647 ,
         176285.46799103,  -76457.40936593,  128275.29064507,
         -35240.8041254 ,   38713.31895115,  -14188.958722  ,
         138192.19414686,   15355.06285137,

In [15]:
(np.transpose(sample, (0, 2, 1))@check_error)[1]

array([[  67568.71473288,  -54068.69574521,   23596.48727212,
        -128521.19845805,    9880.1331249 ,  -36268.42439887,
         -41735.56494009,   56111.89244778,   35586.33672801,
         -34324.09351658,  -32552.19429502,   57874.73568831,
          47993.05864908,   -7400.9907628 ,  -56195.24721095,
         -68185.55195928,  -31738.97832506,   49116.0314091 ,
          -9851.24133631,  -53612.86705739,  -41753.73630245,
         -19076.15984663,   81432.55214513,  -24529.29526251,
          77003.18886427],
       [  17865.53255925,   -5576.68223925,   63146.60126128,
         -22283.36554166,   23351.3610662 ,  -57360.81541882,
         -45170.46704203,  -18070.80811844,   41350.80997168,
          31992.24958086,   87390.00853691,  -10115.6751183 ,
           2225.1500625 ,   22179.83199527,    6592.1886647 ,
         176285.46799103,  -76457.40936593,  128275.29064507,
         -35240.8041254 ,   38713.31895115,  -14188.958722  ,
         138192.19414686,   15355.06285137,

# Softmax (dim=-1) check

In [16]:
class SoftMaxLayer3D(BaseLayer):
    def __init__(self):
        self.input = None
        self.forward_result = None

    def forward(self, x, grad=True):
        self.input = x
        exp = np.exp(x)
        self.forward_result = exp / np.sum(exp, axis=-1, keepdims=True)
        return self.forward_result

    def backward(self, output_error):
        "https://binpord.github.io/2021/09/26/softmax_backprop.html"
        return (output_error - (output_error*self.forward_result).sum(axis=-1, keepdims=True)) * self.forward_result

In [17]:
# проверка того, что softmax работает также как и в torch

sample = np.random.normal(loc=0, scale=100, size=(100, 32, 16))

softmax = SoftMaxLayer3D()

torch_input = torch.tensor(sample, dtype=torch.float64, requires_grad=True)
torch_out = nn.functional.softmax(torch_input, dim=-1)

# проверка что forward работает также как у torch, 
print("Vanilla Forward такой же как и torch forward:", np.allclose(torch_out.detach().numpy(), softmax(sample)))

print("Shape выхода слоя:", softmax(sample).shape)

# случайная ошибка, которая приходит "сверху" от вышестоящих слоев, по размеру она совпадает с выходом слоя
check_error = np.random.normal(loc=-3, scale=100, size=(softmax(sample).shape))
check_error_torch = torch.tensor(check_error)

torch_out.backward(check_error_torch)  # считаем градиенты для всех тензоров, которые участвуют в forward проходе
in_error = softmax.backward(check_error)  # тоже самое, только в ручном слое

# проверка градиентов весов и входа
print("Градиент по входу совпадает:", np.allclose(torch_input.grad.detach().numpy(), in_error))

Vanilla Forward такой же как и torch forward: True
Shape выхода слоя: (100, 32, 16)
Градиент по входу совпадает: True


# attention check

In [18]:
class MultiHeadAttentionLayer(BaseLayer):
    def __init__(self, hid_dim: int, n_heads: int) -> None:
        
        assert hid_dim % n_heads == 0
        
        self.input = None
        self.attn_bias = None
        self.attentions = []
        self.q = None
        self.k = None
        self.v = None
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_size = hid_dim // n_heads
        
        self.c_attn = Linear3d(hid_dim, hid_dim * 3)
        self.c_proj = Linear3d(hid_dim, hid_dim)
        self.softmax = SoftMaxLayer3D()

        self.scale = np.sqrt(self.head_size)

    def set_optimizer(self, optimizer) -> None:
        self.c_attn.set_optimizer(optimizer)
        self.c_proj.set_optimizer(optimizer)

    def forward(self, x: np.array, grad: bool = True) -> np.array:
        self.input = x
        
        q_k_v = self.c_attn(x)
        self.q = q_k_v[:, :, :self.hid_dim]
        self.k = q_k_v[:, :, self.hid_dim:self.hid_dim*2]
        self.v = q_k_v[:, :, self.hid_dim*2:self.hid_dim*3]
        assert self.q.shape == self.k.shape == self.v.shape == x.shape, "q, k and v must have the same shape as x"

        head_outputs = []
        for head_index in range(self.n_heads):
            head_selector = range(self.head_size * head_index, self.head_size * (head_index + 1))

            head_queries = self.q[..., head_selector]
            head_keys = self.k[..., head_selector]
            head_values = self.v[..., head_selector]

            single_head_output = self._attention_for_head(
                head_queries, head_keys, head_values,
                is_causal=True)
            head_outputs.append(single_head_output)

        combined_head_outputs = np.concatenate(head_outputs, axis=-1)
        return self.c_proj(combined_head_outputs)

    def _attention_for_head(self, query, key, value, is_causal=False):
        L, S = query.shape[-2], key.shape[-2]
        self.attn_bias = np.zeros((L, S), dtype=query.dtype)
        if is_causal:
            temp_mask = np.tril(np.ones((L, S)))
            self.attn_bias = np.where(temp_mask==0, float('-inf'), 0)
            self.attn_bias = self.attn_bias.astype(query.dtype)
    
        attn_weight = query @ key.transpose(0, 2, 1) / self.scale
        attn_weight += self.attn_bias
        
        attn_weight = self.softmax(attn_weight)
        self.attentions.append(attn_weight)
        
        result = attn_weight @ value

        # result = self.softmax((query @ key.transpose(0, 2, 1) / self.scale) + self.attn_bias) @ value
        
        return result

    def _attention_for_head_grad(self, query, key, value, output_error, head_n):
        # looking at the result expression in the forward pass
        cur_attention = self.attentions[head_n]
        
        v_grad = cur_attention.transpose((0, 2, 1)) @ output_error
        input_error = output_error @ value.transpose((0, 2, 1))

        # we use layer, which uses it's states to calculate grad, so we need to set state that was used during forward pass in current head
        self.softmax.forward_result = cur_attention
        softmax_grad = self.softmax.backward(input_error)

        k_grad = softmax_grad.transpose(0, 2, 1) @ query / self.scale # we need to transpose output error due to the fact key was transposed in forward
        q_grad = softmax_grad @ key / self.scale

        return q_grad, k_grad, v_grad

    def backward(self, output_error: np.array) -> np.array:
        c_proj_w_grad, c_proj_b_grad, projection_error = self.c_proj.backward(output_error)
        
        q_grads, k_grads, v_grads = [], [], []
        for head_index in range(self.n_heads):
            # for each head we choose it's error
            head_selector = range(self.head_size * head_index, self.head_size * (head_index + 1))

            attention_error = projection_error[..., head_selector]
            head_queries = self.q[..., head_selector]
            head_keys = self.k[..., head_selector]
            head_values = self.v[..., head_selector]

            q_grad, k_grad, v_grad = self._attention_for_head_grad(head_queries, head_keys, head_values, attention_error, head_index)
            q_grads.append(q_grad)
            k_grads.append(k_grad)
            v_grads.append(v_grad)

        q_grads = np.concatenate(q_grads, axis=-1)
        k_grads = np.concatenate(k_grads, axis=-1)
        v_grads = np.concatenate(v_grads, axis=-1)

        q_k_v_output = np.concatenate([q_grads, k_grads, v_grads], axis=-1)
        
        c_attn_w_grad, c_attn_b_grad, input_error = self.c_attn.backward(q_k_v_output)
        return input_error

In [19]:
# torch attention
class MaskedSelfAttention(nn.Module):
    def __init__(self, dim: int, num_heads: int):
        super().__init__()
        self.c_attn = nn.Linear(dim, dim * 3)  # query + key + value, combined
        self.c_proj = nn.Linear(dim, dim)  # output projection
        self.dim, self.num_heads = dim, num_heads
        self.head_size = dim // num_heads

        self.scale = torch.sqrt(torch.FloatTensor([self.head_size]))

    # you can choose less effective method and uncomment the code below, it works the same
    # ------------- start of less effective -------------

    # def forward(self, x):
    #     q, k, v = self.c_attn(x).split(dim=-1, split_size=self.dim)
    #     assert q.shape == k.shape == v.shape == x.shape, "q, k and v must have the same shape as x"


    #     # Note: this is an inefficient implementation that uses a for-loop.
    #     # To get the full grade during homework, please re-implement this code:
    #     # 1) do not use for-loops (or other loops). Compute everything in parallel with vectorized operations
    #     # 2) do not use F.scaled_dot_product_attention - write your own attention code using basic PyTorch ops
    #     head_outputs = []
    #     for head_index in range(self.num_heads):
    #         head_selector = range(self.head_size * head_index, self.head_size * (head_index + 1))

    #         head_queries = q[..., head_selector]
    #         head_keys = k[..., head_selector]
    #         head_values = v[..., head_selector]

    #         single_head_output = self.scaled_dot_product_attention(
    #             head_queries, head_keys, head_values,
    #             is_causal=True)
    #         # docs: https://pytorch.org/docs/stable/generated/torch.nn.functional.scaled_dot_product_attention.html
    #         head_outputs.append(single_head_output)

    #     combined_head_outputs = torch.cat(head_outputs, dim=-1)
    #     return self.c_proj(combined_head_outputs)

    # def scaled_dot_product_attention(self, query, key, value, attn_mask=None, dropout_p=0.0, is_causal=False, scale=None) -> torch.Tensor:
    #     # Efficient implementation equivalent to the following:
    #     L, S = query.size(-2), key.size(-2)
    #     scale_factor = 1 / math.sqrt(query.size(-1)) if scale is None else scale
    #     attn_bias = torch.zeros(L, S, dtype=query.dtype)
    #     if is_causal:
    #         assert attn_mask is None
    #         temp_mask = torch.ones(L, S, dtype=torch.bool).tril(diagonal=0)
    #         attn_bias.masked_fill_(temp_mask.logical_not(), float("-inf"))
    #         attn_bias.to(query.dtype)
    
    #     if attn_mask is not None:
    #         if attn_mask.dtype == torch.bool:
    #             attn_bias.masked_fill_(attn_mask.logical_not(), float("-inf"))
    #         else:
    #             attn_bias += attn_mask
    #     attn_weight = query @ key.transpose(-2, -1) * scale_factor
    #     attn_weight += attn_bias
    #     attn_weight = torch.softmax(attn_weight, dim=-1)
    #     # attn_weight = torch.dropout(attn_weight, dropout_p, train=True)
    #     return attn_weight @ value

    # ------------- end of less effective -------------

    def forward(self, x):
        # kinda more effective
        batch_size, seq_len, _ = x.shape
        q, k, v = self.c_attn(x).split(dim=-1, split_size=self.dim)
        # [BATCH_SIZE, SEQ_LEN, DIM]
        assert q.shape == k.shape == v.shape == x.shape, "q, k and v must have the same shape as x"
        
        # [BATCH_SIZE, NUM_HEADS, SEQ_LEN, HEAD_SIZE]
        q = q.view(batch_size, seq_len, self.num_heads, self.head_size).permute(0, 2, 1, 3)
        k = k.view(batch_size, seq_len, self.num_heads, self.head_size).permute(0, 2, 1, 3)
        v = v.view(batch_size, seq_len, self.num_heads, self.head_size).permute(0, 2, 1, 3)

        # [BATCH_SIZE, NUM_HEADS, SEQ_LEN, SEQ_LEN]
        energy = torch.matmul(q, k.permute(0, 1, 3, 2)) / self.scale

        # чтобы не смотреть в будущее
        mask = torch.tril(torch.ones((seq_len, seq_len)))
        energy = energy.masked_fill(mask == 0, float('-inf'))

        # [BATCH_SIZE, NUM_HEADS, SEQ_LEN, SEQ_LEN]
        probs = nn.functional.softmax(energy, dim=-1)

        # [BATCH_SIZE, NUM_HEADS, SEQ_LEN, HEAD_SIZE]
        output = torch.matmul(probs, v)

        # [BATCH_SIZE, SEQ_LEN, NUM_HEADS, HEAD_SIZE]
        output = output.permute(0, 2, 1, 3).contiguous()

        # [BATCH_SIZE, SEQ_LEN, DIM]
        output = output.view(batch_size, -1, self.dim)

        # [BATCH_SIZE, SEQ_LEN, DIM]
        output = self.c_proj(output)
        return output

In [20]:
# проверка того, что multihead masked attention работает также как и в torch

for head_n in [1, 3, 6]:
    print('Число голов:', head_n)
    sample = np.random.randn(100, 32, 36)
    
    attention = MultiHeadAttentionLayer(36, head_n)
    torch_attention = MaskedSelfAttention(36, head_n)
    # ставим одинаковые иходные веса для слоев
    attention.c_attn.w = torch_attention.c_attn.weight.data.numpy().T
    attention.c_attn.b = torch_attention.c_attn.bias.data.numpy().reshape(1, -1)
    attention.c_proj.w = torch_attention.c_proj.weight.data.numpy().T
    attention.c_proj.b = torch_attention.c_proj.bias.data.numpy().reshape(1, -1)

    attention_out = attention(sample)
    torch_input = torch.tensor(sample, dtype=torch.float32, requires_grad=True)
    torch_out = torch_attention(torch_input)
    
    # проверка что forward работает также как у torch, 
    print("Vanilla Forward такой же как и torch forward:", np.allclose(torch_out.detach().numpy(), attention_out, atol=1e-6))
    
    print("Shape выхода слоя:", attention_out.shape)
    
    # случайная ошибка, которая приходит "сверху" от вышестоящих слоев, по размеру она совпадает с выходом слоя
    check_error = np.random.randn(100, 32, 36)
    check_error_torch = torch.tensor(check_error)
    
    torch_out.backward(check_error_torch)  # считаем градиенты для всех тензоров, которые участвуют в forward проходе
    in_error = attention.backward(check_error)  # тоже самое, только в ручном слое
    
    # проверка градиентов весов и входа
    print("Градиент по входу совпадает:", np.allclose(torch_input.grad.detach().numpy(), in_error, atol=1e-6))
    print('='*30)

Число голов: 1
Vanilla Forward такой же как и torch forward: True
Shape выхода слоя: (100, 32, 36)
Градиент по входу совпадает: True
Число голов: 3
Vanilla Forward такой же как и torch forward: True
Shape выхода слоя: (100, 32, 36)
Градиент по входу совпадает: True
Число голов: 6
Vanilla Forward такой же как и torch forward: True
Shape выхода слоя: (100, 32, 36)
Градиент по входу совпадает: True


# Layernorm check

In [21]:
class LayerNorm(BaseLayer):
    def __init__(self, dim, eps=1e-5, elementwise_affine=True, bias=True):
        self.dim = dim
        self.eps = eps
        self.elementwise_affine = elementwise_affine
        self.use_bias = bias

        self.input = None
        self.normalized = None
        self.mean = None
        self.var = None

        self.weight = np.ones(shape=dim)
        self.bias = np.zeros(shape=dim)

    def forward(self, x, grad=True):
        """Only for -1 axis"""
        self.input = x

        self.mean = np.mean(x, axis=-1, keepdims=True)
        self.var = np.var(x, axis=-1, keepdims=True, ddof=0)
        self.x_centered = (x - self.mean)
        self.std = np.sqrt(self.var+self.eps)

        self.normalized = self.x_centered / self.std

        result = self.normalized * self.weight + self.bias
        return result

    def backward(self, output_error):
        weight_grad = None
        bias_grad = None
        if self.elementwise_affine:
            weight_grad = np.sum(output_error * self.normalized, axis=(0, 1))

        if self.use_bias:
            bias_grad = np.sum(output_error, axis=(0, 1))

        # производная по normalized
        dldx = output_error * self.weight
        # производная по дисперсии в знаменателе
        dldsigma2 = np.sum(dldx * self.x_centered * (-1/2) * (self.std ** -3), axis=-1, keepdims=True)
        dldmu = np.sum((dldx * (-1/self.std)), axis=-1, keepdims=True) + dldsigma2\
        * np.sum(-2*self.x_centered, axis=-1, keepdims=True) / self.dim

        input_error = dldx / self.std + dldsigma2 * 2 * self.x_centered / self.dim + dldmu / self.dim
        return weight_grad, bias_grad, input_error

In [22]:
# проверка того, что layernorm работает также как и в torch
batch_size, seq_len, emb_dim = 100, 10, 32
sample = np.random.randn(batch_size, seq_len, emb_dim)

weight = np.random.randn(emb_dim)
bias = np.random.randn(emb_dim)

layernorm = LayerNorm(emb_dim)
layernorm.weight = weight
layernorm.bias = bias

out = layernorm(sample)

torch_input = torch.tensor(sample, dtype=torch.float64, requires_grad=True)
torch_layernorm = nn.LayerNorm(emb_dim)
torch_layernorm.weight.data = torch.tensor(weight, requires_grad=True, dtype=torch.float64)
torch_layernorm.bias.data = torch.tensor(bias, requires_grad=True, dtype=torch.float64)
torch_out = torch_layernorm(torch_input)

# проверка что forward работает также как у torch, 
print("Vanilla Forward такой же как и torch forward:", np.allclose(torch_out.detach().numpy(), out))

print("Shape выхода слоя:", out.shape)

# случайная ошибка, которая приходит "сверху" от вышестоящих слоев, по размеру она совпадает с выходом слоя
check_error = np.random.normal(loc=-3, scale=100, size=out.shape)
check_error_torch = torch.tensor(check_error)

torch_out.backward(check_error_torch)  # считаем градиенты для всех тензоров, которые участвуют в forward проходе
weight_grad, bias_grad, in_error = layernorm.backward(check_error)  # тоже самое, только в ручном слое

# проверка градиентов весов и входа
print("Градиент weight совпадает:", np.allclose(torch_layernorm.weight.grad.detach().numpy(), weight_grad))
print("Градиент bias совпадает:", np.allclose(torch_layernorm.bias.grad.detach().numpy(), bias_grad))
print("Градиент по входу совпадает:", np.allclose(torch_input.grad.detach().numpy(), in_error))

Vanilla Forward такой же как и torch forward: True
Shape выхода слоя: (100, 10, 32)
Градиент weight совпадает: True
Градиент bias совпадает: True
Градиент по входу совпадает: True


# gpt fully connected check

In [23]:
class Activation(BaseLayer):
    """
    Activation class is used for activation function of the FC layer
    Params:
    activation_function - activation function (e.g. sigmoid, RElU, tanh)
    activation_derivative - derivative of the activation function
    Methods:
    forward(x) - performs forward pass of the layer
    backward(output_error, learning_rate) - performs backward pass of the layer
    """

    def __init__(self, activation_function: callable, activation_derivative: callable) -> None:
        super().__init__()
        self.input = None
        self.activation = activation_function
        self.derivative = activation_derivative

    def forward(self, x: np.array, grad: bool = True) -> np.array:
        self.input = x
        return self.activation(x)

    def backward(self, output_error: np.array) -> np.array:
        return output_error * self.derivative(self.input)

def relu(z: Union[np.array, float, int, list]) -> Union[np.array, float, int]:
    """
    ReLU function
    """
    return np.maximum(0, z)


def relu_derivative(z: Union[np.array, float, int, list]) -> np.array:
    """
    ReLU function derivative
    """
    return (z > 0).astype(int)

In [24]:
class MyFullyConnected(BaseLayer):
    def __init__(self, dim):
        self.c_fc = Linear3d(dim, 4*dim)
        self.relu = Activation(relu, relu_derivative)
        self.c_proj = Linear3d(4*dim, dim)

    def forward(self, x, grad=True):
        outputs = self.c_fc(x)

        outputs = self.relu(outputs)

        outputs = self.c_proj(outputs)
        return outputs

    def backward(self, output_error):
        c_proj_w_grad, c_proj_b_grad, c_proj_error = self.c_proj.backward(output_error)
        relu_error = self.relu.backward(c_proj_error)
        c_fc_w_grad, c_fc_b_grad, c_fc_error = self.c_fc.backward(relu_error)

        return c_proj_w_grad, c_proj_b_grad, c_fc_w_grad, c_fc_b_grad, c_fc_error

In [25]:
class FullyConnected(nn.Module):
    def __init__(self, dim: int):
        super().__init__()
        self.c_fc = nn.Linear(dim, 4  * dim)
        self.relu = nn.ReLU()
        self.c_proj = nn.Linear(4 * dim, dim)

    def forward(self, x):
        # x.shape = [batch_size, seq_length, dim]
        outputs = self.c_fc(x)

        outputs = self.relu(outputs)

        outputs = self.c_proj(outputs)
        return outputs

In [26]:
# проверка того, что gpt fully connected работает также как и в torch
batch_size, seq_len, emb_dim = 100, 10, 32
sample = np.random.randn(batch_size, seq_len, emb_dim).astype(np.float32)

my_mlp = MyFullyConnected(emb_dim)
torch_mlp = FullyConnected(emb_dim)
# ставим одинаковые иходные веса для слоев
my_mlp.c_fc.w = torch_mlp.c_fc.weight.data.numpy().T
my_mlp.c_fc.b = torch_mlp.c_fc.bias.data.numpy().reshape(1, -1)
my_mlp.c_proj.w = torch_mlp.c_proj.weight.data.numpy().T
my_mlp.c_proj.b = torch_mlp.c_proj.bias.data.numpy().reshape(1, -1)

out = my_mlp(sample)

torch_input = torch.tensor(sample, dtype=torch.float32, requires_grad=True)
torch_out = torch_mlp(torch_input)

# проверка что forward работает также как у torch, 
print("Vanilla Forward такой же как и torch forward:", np.allclose(torch_out.detach().numpy(), out, atol=1e-6))

print("Shape выхода слоя:", out.shape)

# случайная ошибка, которая приходит "сверху" от вышестоящих слоев, по размеру она совпадает с выходом слоя
check_error = np.random.randn(*out.shape)
check_error_torch = torch.tensor(check_error)

torch_out.backward(check_error_torch)  # считаем градиенты для всех тензоров, которые участвуют в forward проходе
c_proj_weight_grad, c_proj_bias_grad, c_fc_weight_grad, c_fc_bias_grad, c_fc_in_error = my_mlp.backward(check_error)  # тоже самое, только в ручном слое

# проверка градиентов весов и входа
print("Градиент c_proj_weight совпадает:", np.allclose(torch_mlp.c_proj.weight.grad.detach().numpy(), c_proj_weight_grad.T, atol=1e-5))
print("Градиент c_proj_bias совпадает:", np.allclose(torch_mlp.c_proj.bias.grad.detach().numpy(), c_proj_bias_grad, atol=1e-5))
print("Градиент c_fc_weight совпадает:", np.allclose(torch_mlp.c_fc.weight.grad.detach().numpy(), c_fc_weight_grad.T, atol=1e-5))
print("Градиент c_fc_bias совпадает:", np.allclose(torch_mlp.c_fc.bias.grad.detach().numpy(), c_fc_bias_grad, atol=1e-5))

print("Градиент по входу совпадает:", np.allclose(torch_input.grad.detach().numpy(), c_fc_in_error, atol=1e-5))

Vanilla Forward такой же как и torch forward: True
Shape выхода слоя: (100, 10, 32)
Градиент c_proj_weight совпадает: True
Градиент c_proj_bias совпадает: True
Градиент c_fc_weight совпадает: True
Градиент c_fc_bias совпадает: True
Градиент по входу совпадает: True


# gpt transformer check

In [27]:
class TransformerLayer(nn.Module):
    def __init__(self, dim: int, num_heads: int):
        super().__init__()
        self.ln_1 = nn.LayerNorm(dim)
        self.attn = MaskedSelfAttention(dim, num_heads)
        self.ln_2 = nn.LayerNorm(dim)
        self.mlp = FullyConnected(dim)

    def forward(self, x):
        output1 = self.ln_1(x)
        output1 = self.attn(output1)
        output1 = output1 + x

        output2 = self.ln_2(output1)
        output2 = self.mlp(output2)
        output = output2 + output1
        return output

In [28]:
class MyTransformer(BaseLayer):
    def __init__(self, dim: int, num_heads: int):
        self.ln_1 = LayerNorm(dim)
        self.attn = MultiHeadAttentionLayer(dim, num_heads)
        self.ln_2 = LayerNorm(dim)
        self.mlp = MyFullyConnected(dim)

    def forward(self, x, grad=True):
        output1 = self.ln_1(x)
        output1 = self.attn(output1)
        output1 = output1 + x

        output2 = self.ln_2(output1)
        output2 = self.mlp(output2)
        output = output2 + output1
        return output

    def backward(self, output_error):
        # output = output1 + output2 + x
        # doutput_doutput2 = 1 и надо это умножить на output_error
        c_proj_w_grad, c_proj_b_grad, c_fc_w_grad, c_fc_b_grad, mlp_error = self.mlp.backward(output_error)
        _, _, ln2_error = self.ln_2.backward(mlp_error)
        # doutput_doutput1 = 1 и надо это умножить на output_error + ошибка, которая пришла выше
        attn_error = self.attn.backward(ln2_error+output_error) 
        ln1_weight_grad, _, ln1_error = self.ln_1.backward(attn_error)
        # doutput_dx = 1 и надо это умножить на output_error + ошибка, которая пришла выше
        return c_proj_w_grad, ln1_weight_grad, ln1_error+ln2_error+output_error

In [29]:
# проверка того, что gpt работает также как и в torch
batch_size, seq_len, emb_dim = 100, 10, 32
sample = np.random.randn(batch_size, seq_len, emb_dim).astype(np.float32)

my_transformer = MyTransformer(emb_dim, 8)
torch_transformer = TransformerLayer(emb_dim, 8)

# ставим одинаковые иходные веса для слоев, layernorm инциализируются одинаково
my_transformer.attn.c_attn.w = torch_transformer.attn.c_attn.weight.data.numpy().T
my_transformer.attn.c_attn.b = torch_transformer.attn.c_attn.bias.data.numpy()
my_transformer.attn.c_proj.w = torch_transformer.attn.c_proj.weight.data.numpy().T
my_transformer.attn.c_proj.b = torch_transformer.attn.c_proj.bias.data.numpy()

my_transformer.mlp.c_fc.w = torch_transformer.mlp.c_fc.weight.data.numpy().T
my_transformer.mlp.c_fc.b = torch_transformer.mlp.c_fc.bias.data.numpy()
my_transformer.mlp.c_proj.w = torch_transformer.mlp.c_proj.weight.data.numpy().T
my_transformer.mlp.c_proj.b = torch_transformer.mlp.c_proj.bias.data.numpy()

out = my_transformer(sample)

torch_input = torch.tensor(sample, dtype=torch.float32, requires_grad=True)
torch_out = torch_transformer(torch_input)

# проверка что forward работает также как у torch, 
print("Vanilla Forward такой же как и torch forward:", np.allclose(torch_out.detach().numpy(), out, atol=1e-6))

print("Shape выхода слоя:", out.shape)

# # случайная ошибка, которая приходит "сверху" от вышестоящих слоев, по размеру она совпадает с выходом слоя
check_error = np.random.randn(*out.shape)
check_error_torch = torch.tensor(check_error)

torch_out.backward(check_error_torch)  # считаем градиенты для всех тензоров, которые участвуют в forward проходе
c_proj_w_grad, ln1_weight_grad, in_error = my_transformer.backward(check_error)  # тоже самое, только в ручном слое

# проверка градиентов весов и входа
print("Градиент mlp c_proj_weight совпадает:", np.allclose(torch_transformer.mlp.c_proj.weight.grad.detach().numpy(), c_proj_w_grad.T, atol=1e-5))
print("Градиент ln1 weight совпадает:", np.allclose(torch_transformer.ln_1.weight.grad.detach().numpy(), ln1_weight_grad, atol=1e-5))

print("Градиент по входу совпадает:", np.allclose(torch_input.grad.detach().numpy(), in_error, atol=1e-6))

Vanilla Forward такой же как и torch forward: True
Shape выхода слоя: (100, 10, 32)
Градиент mlp c_proj_weight совпадает: True
Градиент ln1 weight совпадает: True
Градиент по входу совпадает: True


In [30]:
torch_input.grad.detach().numpy(), in_error

(array([[[ 1.2432584 ,  0.16611727, -1.6155889 , ..., -0.45531756,
          -0.79728395, -0.29408318],
         [-1.3477178 ,  1.2574115 , -2.1409097 , ..., -1.3946066 ,
           0.23250823, -0.51982355],
         [-0.52904916, -0.01775089, -0.05373952, ..., -2.4188576 ,
          -1.1829457 ,  1.1646185 ],
         ...,
         [ 0.43370864, -0.22322391, -0.8629938 , ..., -0.52332103,
           0.62400216,  0.7159316 ],
         [-3.0608191 , -0.4776267 , -0.9876322 , ..., -0.20581292,
          -0.8383774 ,  1.4449457 ],
         [ 0.2463032 , -0.10206585, -0.07061028, ...,  0.14600801,
          -0.8268822 , -1.879227  ]],
 
        [[-1.3731863 ,  1.2201213 ,  1.451661  , ..., -0.4690495 ,
          -0.83982235, -1.0552561 ],
         [ 1.6166644 ,  0.82144153, -0.4345418 , ...,  0.6809467 ,
           0.2236872 ,  0.7280807 ],
         [-0.7798389 ,  0.450372  ,  1.6508105 , ..., -0.32078117,
           0.4679607 , -0.38489607],
         ...,
         [ 0.53584814,  1.1203095

# conv2d check

In [31]:
class Conv2D(BaseLayer):
    def __init__(self, in_channels: int, out_channels: int, kernel_size: int, stride: int, padding: int = 0) -> None:
        super().__init__()
        self.in_channels = in_channels  # number of input channels
        self.out_channels = out_channels  # number of output channels
        self.kernel_size = kernel_size  # kernel size, int
        self.stride = stride  # stride, int
        self.padding = padding  # padding, int, only zeros padding

        limit = 1 / np.sqrt(self.kernel_size ** 2)
        self.kernel = np.random.uniform(
            -limit,
            limit,
            size=(self.out_channels, self.in_channels, self.kernel_size, self.kernel_size)
        )

        self.bias = np.zeros((self.out_channels, 1))
        self.kernel_optimizer = None
        self.bias_optimizer = None

        self.input_tensor = None
        self.batch_size = None
        self.reformatted_input = None
        self.reformatted_kernel = None

        self.channel_steps = []

    def set_kernel(self, kernel: np.array) -> None:
        """
        For setting the kernel manually
        """
        assert len(kernel.shape) == 4
        self.kernel = kernel

    def set_optimizer(self, optimizer) -> None:
        self.kernel_optimizer = copy.copy(optimizer)
        self.bias_optimizer = copy.copy(optimizer)

        self.kernel_optimizer.set_weight(self.kernel)
        self.bias_optimizer.set_weight(self.bias)

    def _reformat_kernel(self) -> np.array:
        """
        Reformat the kernel to perform convolution as matrix multiplication
        """
        assert len(self.kernel.shape) == 4
        return self.kernel.reshape((self.out_channels, -1))

    def _reformat_input(self, input_tensor: np.array) -> np.array:
        """
        Reformat the batch of input images to perform convolution as matrix multiplication
        """
        result = []
        for image in input_tensor:
            result.append(self._reformat_image(image))

        return np.hstack(result)

    def _reformat_image(self, image: np.array) -> np.array:
        """
        Reformatting each image in the batch
        """
        result = []
        for channel in image:
            result.append(self._reformat_channel(channel))

        return np.vstack(result)

    def _reformat_channel(self, channel: np.array) -> np.array:
        """
        Reformat each channel in the image with addinction of zeros as padding
        """
        input_map = self._add_padding(channel)

        self.padded_width_in, self.padded_height_in = input_map.shape

        width_in, height_in = channel.shape

        self.width_out = (width_in - self.kernel_size + 2 * self.padding) // self.stride + 1
        self.height_out = (height_in - self.kernel_size + 2 * self.padding) // self.stride + 1

        output_map = []

        write_steps = True
        if self.channel_steps:
            write_steps = False

        for i in range(self.width_out):
            for j in range(self.height_out):
                horizontal_slice_from = i * self.stride
                horizontal_slice_to = self.kernel_size + i * self.stride

                vertical_slice_from = j * self.stride
                vertical_slice_to = self.kernel_size + j * self.stride

                if write_steps:
                    self.channel_steps.append(
                        (horizontal_slice_from, horizontal_slice_to, vertical_slice_from, vertical_slice_to)
                    )

                subsample = input_map[horizontal_slice_from:horizontal_slice_to, vertical_slice_from:vertical_slice_to]
                output_map.append(subsample.reshape(-1, 1))

        return np.hstack(output_map)

    def _add_padding(self, input_tensor: np.array) -> np.array:
        new_width = input_tensor.shape[0] + self.padding * 2
        new_height = input_tensor.shape[1] + self.padding * 2
        padded_map = np.zeros((new_width, new_height))

        padded_map[self.padding:new_width - self.padding, self.padding:new_height - self.padding] = input_tensor
        return padded_map

    def _reformat_forward(self, forward_output: np.array) -> np.array:
        result = []
        for image in forward_output:
            result.append(image.reshape(self.batch_size, self.height_out, self.width_out))
        return np.hstack(result).reshape((self.batch_size, self.out_channels, self.height_out, self.width_out))

    @staticmethod
    def _reformat_output(output_error: np.array) -> np.array:
        images_result = []
        for image in output_error:
            channel_result = []
            for channel in image:
                channel_result.append(channel.ravel())
            images_result.append(np.vstack(channel_result))
        return np.hstack(images_result)

    def _format_image_back(self, image: np.array) -> np.array:
        zeros = np.zeros((self.in_channels, self.padded_width_in, self.padded_height_in))
        elements_in_kernel = self.kernel_size ** 2
        for i in range(self.in_channels):
            for j, line in enumerate(image.T):
                reshaped_conv = line[i * elements_in_kernel:(i + 1) * elements_in_kernel].reshape(self.kernel_size,
                                                                                                  self.kernel_size)

                horizontal_slice_from, horizontal_slice_to, vertical_slice_from, vertical_slice_to = self.channel_steps[
                    j]
                zeros[i][horizontal_slice_from:horizontal_slice_to,
                         vertical_slice_from:vertical_slice_to] += reshaped_conv
        return zeros

    def _cut_padding(self, image: np.array) -> np.array:
        result = []
        for channel in image:
            channel_no_pad = channel[self.padding:self.padded_width_in - self.padding,
                                     self.padding:self.padded_height_in - self.padding]
            result.append(channel_no_pad[np.newaxis, :])
        return np.vstack(result)

    def _reformat_input_error(self, input_error: np.array) -> np.array:
        assert len(input_error.T) % self.batch_size == 0
        size_for_one_image = int(len(input_error.T) / self.batch_size)
        result = []
        for i in range(self.batch_size):
            image = input_error[:, i * size_for_one_image:(i + 1) * size_for_one_image]
            image = self._format_image_back(image)

            if self.padding != 0:
                image = self._cut_padding(image)

            result.append(image[np.newaxis, :])
        return np.vstack(result)

    def forward(self, input_tensor: np.array, grad: bool = False) -> np.array:
        assert len(input_tensor.shape) == 4
        self.input_tensor = input_tensor
        self.batch_size = self.input_tensor.shape[0]

        self.reformatted_input = self._reformat_input(input_tensor)
        self.reformatted_kernel = self._reformat_kernel()

        result = self.reformatted_kernel.dot(self.reformatted_input) + self.bias
        return self._reformat_forward(result)

    def backward(self, output_error: np.array) -> np.array:
        # assert self.kernel_optimizer is not None and self.bias_optimizer is not None, 'You should set an optimizer'
        output_error = self._reformat_output(output_error)
        w_grad = output_error.dot(self.reformatted_input.T).reshape(self.kernel.shape)
        b_grad = output_error.dot(np.ones((output_error.shape[1], 1)))

        # self.kernel = self.kernel_optimizer.step(w_grad)
        # self.bias = self.bias_optimizer.step(b_grad)

        input_error = self.reformatted_kernel.T.dot(output_error)#.reshape(self.input_tensor.shape)
        input_error = self._reformat_input_error(input_error)
        return w_grad, b_grad, input_error

In [32]:
# проверка того, что conv2d работает также как и в torch
batch_size, channels_in, height, width = 100, 10, 32, 32

params = [{'stride': 1, 'padding': 0}, {'stride': 3, 'padding': 4}, {'stride': 2, 'padding': 2}]

for param in params:
    print('Ставим параметры stride и padding:', param)
    sample = np.random.randn(batch_size, channels_in, height, width)

    my_conv = Conv2D(in_channels=channels_in, out_channels=8, kernel_size=4, **param)
    torch_conv = nn.Conv2d(in_channels=channels_in, out_channels=8, kernel_size=4, **param)
    
    # ставим одинаковые иходные веса для слоев, layernorm инциализируются одинаково
    torch_conv.weight.data = torch.tensor(my_conv.kernel, requires_grad=True)
    torch_conv.bias.data = torch.tensor(my_conv.bias.ravel(), requires_grad=True)
    
    out = my_conv(sample)
    
    torch_input = torch.tensor(sample, requires_grad=True)
    torch_out = torch_conv(torch_input)
    
    # проверка что forward работает также как у torch, 
    print("Vanilla Forward такой же как и torch forward:", np.allclose(torch_out.detach().numpy(), out, atol=1e-6))
    
    print("Shape выхода слоя:", out.shape)
    
    # # случайная ошибка, которая приходит "сверху" от вышестоящих слоев, по размеру она совпадает с выходом слоя
    check_error = np.random.randn(*out.shape)
    check_error_torch = torch.tensor(check_error)
    
    torch_out.backward(check_error_torch)  # считаем градиенты для всех тензоров, которые участвуют в forward проходе
    w_grad, b_grad, input_error = my_conv.backward(check_error)  # тоже самое, только в ручном слое
    
    # проверка градиентов весов и входа
    print("Градиент weight совпадает:", np.allclose(w_grad, torch_conv.weight.grad.data.detach().numpy()))
    print("Градиент bias совпадает:", np.allclose(b_grad.ravel(), torch_conv.bias.grad.data.detach().numpy()))
    
    print("Градиент по входу совпадает:", np.allclose(input_error, torch_input.grad.data.detach().numpy()))
    print('='*30)

Ставим параметры stride и padding: {'stride': 1, 'padding': 0}
Vanilla Forward такой же как и torch forward: True
Shape выхода слоя: (100, 8, 29, 29)
Градиент weight совпадает: True
Градиент bias совпадает: True
Градиент по входу совпадает: True
Ставим параметры stride и padding: {'stride': 3, 'padding': 4}
Vanilla Forward такой же как и torch forward: True
Shape выхода слоя: (100, 8, 13, 13)
Градиент weight совпадает: True
Градиент bias совпадает: True
Градиент по входу совпадает: True
Ставим параметры stride и padding: {'stride': 2, 'padding': 2}
Vanilla Forward такой же как и torch forward: True
Shape выхода слоя: (100, 8, 17, 17)
Градиент weight совпадает: True
Градиент bias совпадает: True
Градиент по входу совпадает: True


# maxpool2d check

In [38]:
class MaxPool2D(BaseLayer):
    def __init__(self, kernel_size: int, stride: int, padding: int = 0) -> None:
        super().__init__()
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

        self.input_tensor = None
        self.channel_steps = []
        self.grad = None
        self.padded_width_in = None
        self.padded_height_in = None

    def _reformat_input(self, input_tensor: np.array) -> np.array:
        result = []
        grad_result = []
        for image in input_tensor:
            forward_result, grad = self._reformat_image(image)
            result.append(forward_result)
            grad_result.append(grad)

        self.grad = grad_result
        return np.vstack(result)

    def _reformat_image(self, image: np.array) -> np.array:
        result = []
        grad_result = []
        for channel in image:
            forward_result, grad = self._reformat_channel(channel)
            result.append(forward_result)
            grad_result.append(grad)

        return np.vstack(result), grad_result

    def _reformat_channel(self, channel: np.array) -> Tuple[np.array, np.array]:
        input_map = self._add_padding(channel)

        width_in, height_in = channel.shape

        self.width_out = (width_in - self.kernel_size + 2 * self.padding) // self.stride + 1
        self.height_out = (height_in - self.kernel_size + 2 * self.padding) // self.stride + 1

        output_map = np.zeros((self.height_out, self.width_out))

        grad = []

        write_steps = True
        if self.channel_steps:
            write_steps = False

        for i in range(self.width_out):
            for j in range(self.height_out):
                horizontal_slice_from = i * self.stride
                horizontal_slice_to = self.kernel_size + i * self.stride

                vertical_slice_from = j * self.stride
                vertical_slice_to = self.kernel_size + j * self.stride

                if write_steps:
                    self.channel_steps.append(
                        (horizontal_slice_from, horizontal_slice_to, vertical_slice_from, vertical_slice_to)
                    )

                subsample = input_map[horizontal_slice_from:horizontal_slice_to, vertical_slice_from:vertical_slice_to]
                output_map[i][j] = np.max(subsample)

                grad.append(np.argmax(subsample))

        return output_map, grad

    def _add_padding(self, input_tensor: np.array) -> np.array:
        self.padded_width_in = input_tensor.shape[0] + self.padding * 2
        self.padded_height_in = input_tensor.shape[1] + self.padding * 2
        padded_map = np.full((self.padded_width_in, self.padded_height_in), float('-inf'))

        padded_map[self.padding:self.padded_width_in - self.padding, self.padding:self.padded_height_in - self.padding] = input_tensor
        return padded_map

    def _rebuild_output_error(self, output_error: np.array) -> np.array:
        batch_size, channels_num, = self.input_tensor.shape[0], self.input_tensor.shape[1]
        zeros = np.zeros((batch_size, channels_num, self.padded_height_in, self.padded_width_in))

        input_error = []
        for image_num, image in enumerate(output_error):
            for channel_num, channel in enumerate(image):
                for i in range(self.width_out * self.height_out):
                    horizontal_slice_from, horizontal_slice_to, vertical_slice_from, vertical_slice_to \
                        = self.channel_steps[i]

                    slice_ = zeros[image_num][channel_num][horizontal_slice_from:horizontal_slice_to,
                                                           vertical_slice_from:vertical_slice_to]
                    cur_max = self.grad[image_num][channel_num][i]

                    flatten_channel = channel.flatten()
                    flatten_zeros = slice_.flatten()

                    flatten_zeros[cur_max] += flatten_channel[i] # !!!

                    slice_ = flatten_zeros.reshape(self.kernel_size, self.kernel_size)
                    zeros[image_num][channel_num][horizontal_slice_from:horizontal_slice_to,
                                                  vertical_slice_from:vertical_slice_to] = slice_

            if self.padding != 0:
                input_error_image = self._cut_padding(zeros[image_num])
            else:
                input_error_image = zeros[image_num]
            input_error.append(input_error_image)

        input_error = np.vstack(input_error).reshape(self.input_tensor.shape)
        return input_error

    def _cut_padding(self, image: np.array) -> np.array:
        result = []
        for channel in image:
            channel_no_pad = channel[self.padding:self.padded_width_in - self.padding,
                                     self.padding:self.padded_height_in - self.padding]
            result.append(channel_no_pad[np.newaxis, :])
        return np.vstack(result)

    def forward(self, input_tensor: np.array, grad: bool = False) -> np.array:
        assert len(input_tensor.shape) == 4
        self.input_tensor = input_tensor

        result = self._reformat_input(input_tensor)
        return result.reshape((input_tensor.shape[0], input_tensor.shape[1], self.height_out, self.width_out))

    def backward(self, output_error: np.array) -> np.array:
        return self._rebuild_output_error(output_error)

In [39]:
# проверка того, что maxpool2d работает также как и в torch
batch_size, channels_in, height, width = 100, 10, 32, 32

params = [
    {'kernel_size': 2, 'stride': 1, 'padding': 0},
    {'kernel_size': 4, 'stride': 3, 'padding': 2},
    {'kernel_size': 4, 'stride': 2, 'padding': 2},
    {'kernel_size': 2, 'stride': 2, 'padding': 0}
]

for param in params:
    print('Ставим параметры:', param)
    sample = np.random.randn(batch_size, channels_in, height, width)

    my_pool = MaxPool2D(**param)
    torch_pool = nn.MaxPool2d(**param)
    
    out = my_pool(sample)
    
    torch_input = torch.tensor(sample, requires_grad=True)
    torch_out = torch_pool(torch_input)
    
    # проверка что forward работает также как у torch, 
    print("Vanilla Forward такой же как и torch forward:", np.allclose(torch_out.detach().numpy(), out, atol=1e-6))
    
    print("Shape выхода слоя:", out.shape)
    
    # # случайная ошибка, которая приходит "сверху" от вышестоящих слоев, по размеру она совпадает с выходом слоя
    check_error = np.random.randn(*out.shape)
    check_error_torch = torch.tensor(check_error)
    
    torch_out.backward(check_error_torch)  # считаем градиенты для всех тензоров, которые участвуют в forward проходе
    input_error = my_pool.backward(check_error)  # тоже самое, только в ручном слое
    
    # проверка градиентов входа
    print("Градиент по входу совпадает:", np.allclose(input_error, torch_input.grad.data.detach().numpy()))
    print('='*30)

Ставим параметры: {'kernel_size': 2, 'stride': 1, 'padding': 0}
Vanilla Forward такой же как и torch forward: True
Shape выхода слоя: (100, 10, 31, 31)
Градиент по входу совпадает: True
Ставим параметры: {'kernel_size': 4, 'stride': 3, 'padding': 2}
Vanilla Forward такой же как и torch forward: True
Shape выхода слоя: (100, 10, 11, 11)
Градиент по входу совпадает: True
Ставим параметры: {'kernel_size': 4, 'stride': 2, 'padding': 2}
Vanilla Forward такой же как и torch forward: True
Shape выхода слоя: (100, 10, 17, 17)
Градиент по входу совпадает: True
Ставим параметры: {'kernel_size': 2, 'stride': 2, 'padding': 0}
Vanilla Forward такой же как и torch forward: True
Shape выхода слоя: (100, 10, 16, 16)
Градиент по входу совпадает: True
